# ML Feature enginerring - SelectKbest

In [1]:
%matplotlib inline
%run talibref.py
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

### Get data

In [26]:
#df=pd.read_csv("data/IYZ.csv")
ticker = 'ITB'
startdate=datetime.date(2010, 1, 1)
enddate=datetime.date.today()
df = generate_ticker_data(ticker, startdate, enddate)

Available data: Ticker(ITB) from 2006.05.05 to 2015.11.27
Usable data: Ticker(ITB) from 2007.02.22 to 2015.11.27 
Returned data: Ticker(ITB) from 2010.01.04 to 2015.11.27 
Save path: data/ITB_from_2010.01.04_2015.11.27.csv


In [27]:
dftouse=df.copy()

### Feature Engineering

In [28]:
IGNORE = ['date', 'result_1','close_1','perf_1','result_14','close_14','perf_14','results']

In [29]:
INDICATORS=[]
for v in df.columns:
    l=df[v].unique()
    if len(l) <= 10 and v not in IGNORE:
        #print v, l
        INDICATORS.append(v)

In [30]:
STANDARDIZABLE = []
for v in df.columns:
    if v not in INDICATORS and v not in IGNORE:
        #print v
        STANDARDIZABLE.append(v)

In [31]:
dftouse['date'] = pd.to_datetime(dftouse['date'])
mask = (dftouse.date < '2015-01-01').values
mask.shape, mask.sum()

((1487,), 1258)

#### 1.2 Standardize the data

Use the mask to compute the training and test parts of the dataframe. Use `StandardScaler` from `sklearn.preprocessing` to "fit" the columns in `STANDARDIZABLE` on the training set. Then use the resultant estimator to transform both the training and the test parts of each of the columns in the dataframe, replacing the old unstandardized values in the `STANDARDIZABLE` columns of `dftouse` by the new standardized ones.

In [32]:
#your code here
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(dftouse[mask][STANDARDIZABLE])
dftouse[STANDARDIZABLE] = scaler.transform(dftouse[STANDARDIZABLE])
dftouse.head()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2010-01-04,-1.055232,-1.056582,-1.042337,-1.030790,-0.616430,12.27,True,0.009046,12.15,False,-0.000822,1,-1.112213,-1.099606,-1.080306,0.800161,0.088232,False,False,False,1,-1.064873,-1.053642,-1.028230,1,1,-0.996363,-1.036326,-1.118015,1,1,-1.087456,1,-0.763132,-0.447355,-0.952848,False,True,0,0.571202,1,0.417710,0,0.158613,-0.039646,0.587714,1,0.770558,1,2.155632,0,0.666324,0,0.576373,0,0.772608,0,0.764450,0,0.361341,1,-0.865794,-0.655599,0,1.180050,-0.415780,0,-0.699566,0,-1.043468,-1.054074,-0.909982,-1.166004,-0.876922,-1.160626,-0.723936,0
1,2010-01-05,-1.023336,-1.036093,-1.044227,-1.010147,-0.878796,12.26,False,-0.000815,12.16,False,-0.008965,1,-1.100090,-1.092604,-1.078782,0.917897,0.173550,False,False,False,1,-1.061762,-1.052063,-1.027100,1,1,-0.996236,-1.035286,-1.115500,1,1,-1.075170,1,-0.704336,-0.317347,-1.181922,False,True,0,0.571202,1,0.525116,0,0.210364,0.013771,0.594222,1,0.844688,1,2.176495,0,0.945762,0,0.798681,0,1.218399,0,1.068674,0,0.409022,1,-0.031562,-0.625546,0,1.180050,0.501045,0,-0.663429,0,-1.030332,-1.029722,-0.883741,-1.153940,-0.862829,-1.138260,-0.696166,0
2,2010-01-06,-1.010202,-1.028642,-0.995105,-1.012024,-0.956545,12.97,True,0.057912,12.20,False,-0.004894,1,-1.090567,-1.084751,-1.072744,0.795378,0.198502,False,False,False,1,-1.058849,-1.050532,-1.026002,1,1,-0.996158,-1.034116,-1.113499,1,1,-1.064112,1,-0.629103,-0.288024,-1.268094,False,True,0,0.571202,1,0.762325,0,0.242766,0.063485,0.552872,1,0.907703,1,2.174899,0,0.676140,0,0.766219,0,1.074731,0,1.021338,0,0.389190,1,0.892584,-0.000569,0,1.040781,1.430617,0,-0.625537,0,-1.012193,-0.996094,-0.847503,-1.137281,-0.843366,-1.107375,-0.657817,0
3,2010-01-07,-0.982058,-0.892667,-0.949762,-0.878780,-0.460615,12.98,True,0.000771,12.34,False,-0.048574,0,-1.054919,-1.068570,-1.077332,1.814589,0.523193,False,True,False,1,-1.050700,-1.047127,-1.023448,1,1,-0.993258,-1.030532,-1.109175,1,1,-1.054160,1,-0.316392,1.752071,-1.649977,False,True,0,1.023908,1,2.277120,0,0.460397,0.150145,0.965456,1,1.084186,1,2.264355,0,1.112054,0,1.849200,0,2.261381,0,1.280418,0,0.505646,1,1.438334,0.836531,0,1.180050,1.430617,0,-0.577834,0,-0.907109,-0.801281,-0.763156,-0.925308,-0.865513,-0.821418,-0.701455,0
4,2010-01-08,-0.878864,-0.900118,-0.872301,-0.876903,-0.578370,12.95,False,-0.002311,12.35,False,-0.048536,0,-1.026854,-1.052295,-1.073670,1.543695,0.712971,False,True,False,1,-1.042794,-1.043740,-1.020900,1,1,-0.989863,-1.026844,-1.105159,1,1,-1.028427,1,-0.026017,1.494864,-1.724919,False,True,0,1.023908,1,2.031605,0,0.625489,0.255041,1.169113,1,1.244347,1,2.316430,0,1.094578,0,1.860484,0,2.401756,0,1.301636,0,0.607788,1,1.417438,1.362959,0,1.180050,1.430617,0,-0.522454,0,-0.883340,-0.757217,-0.715671,-0.903479,-0.840011,-0.780947,-0.651205,1


We create a list `lcols` of the columns we will use in our classifier. This list should not contain the response `RESP`. How many features do we have?

In [33]:
#lcols=list(dftouse.columns)
#lcols.remove(u'results')
lcols=[]
for c in list(dftouse.columns):
    if c not in IGNORE: 
        lcols.append(c)
print len(lcols)

70


### EDA for the data

We create a variable `ccols` which contains all variables not in our indicators list

In [34]:
ccols=[]
for c in lcols:
    if c not in INDICATORS and c not in IGNORE:
        ccols.append(c)
print len(ccols), len(INDICATORS)

44 26


In [35]:
def cv_optimize(clf, parameters, X, y, n_folds, score_func):
    fitmodel = GridSearchCV(clf, param_grid=parameters, cv=n_folds, scoring=score_func)
    fitmodel.fit(X, y)
    return fitmodel.best_estimator_

In [36]:
from sklearn.metrics import confusion_matrix
def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    if mask !=None:
        #print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        #print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    #print "############# based on standard predict ################"
    #print "Accuracy on training data: %0.2f" % (training_accuracy)
    #print "Accuracy on test data:     %0.2f" % (test_accuracy)
    #print confusion_matrix(ytest, clf.predict(Xtest))
    #print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

In [37]:
X=dftouse[lcols].values
y=dftouse['results'].values
Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]  
reuse_split=dict(Xtrain=Xtrain, Xtest=Xtest, ytrain=ytrain, ytest=ytest)

In [38]:
print "whole data set", dftouse['results'].mean()
print "training set", dftouse['results'][mask].mean(), "test set", dftouse['results'][~mask].mean()

whole data set 0.486213853396
training set 0.484101748808 test set 0.497816593886


#####our data is not very asymmetric, but we might still want to balance the trainset

### Balancing train set to test set for training, 
### intended to be used with SVM only

In [39]:
jtrain=np.arange(0, ytrain.shape[0])
n_pos=len(jtrain[ytrain==1])
n_neg=len(jtrain[ytrain==0])
print n_pos, n_neg

ineg = np.random.choice(jtrain[ytrain==0], n_pos, replace=False)
alli=np.concatenate((jtrain[ytrain==1], ineg))
Xtrain_new = Xtrain[alli]
ytrain_new = ytrain[alli]
print Xtrain_new.shape, ytrain_new.shape

reuse_split_balanced=dict(Xtrain=Xtrain_new, Xtest=Xtest, ytrain=ytrain_new, ytest=ytest)

609 649
(1218, 70) (1218,)


## Test all classifiersres using SelectKbest

In [40]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LogisticRegression

def evaluate(clf):
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    df_pred = df[~mask].reset_index(drop=True)
    df_pred['pred_result'] = clf.predict(Xtest)
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])
    _,_,ROI_base,_,_ = evaluate_profit(df_pred, startdate, enddate, 10000, 'result_baseline', 'close', False, [1])
    _,_,ROI_long,_,_ = evaluate_profit(df_pred, startdate, enddate, 10000, 'pred_result', 'close', False, [1])
    _,_,ROI_lgst,_,_ = evaluate_profit(df_pred, startdate, enddate, 10000, 'pred_result', 'close', False, [1,0])
    return training_accuracy, test_accuracy, ROI_base, ROI_long, ROI_lgst

def print_result(clfpipe):     
    print "Number of features: {0}".format(clfpipe.get_params()['selectk__k'])
    print "Features: {0}".format(np.array(lcols)[clfpipe.named_steps['selectk'].get_support()].tolist())
    r = evaluate(clfpipe)
    print "train accuracy: {0}".format(r[0])
    print "test accuracy: {0}".format(r[1])
    print "ROI baseline: {0}".format(r[2])
    print "ROI long-only: {0}".format(r[3])
    print "ROI long-short: {0}".format(r[4])

In [41]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score

In [42]:
%%time
max_number_featrues = 10 
Long_ROI_result={}
print "#############====================== Log Regression =====================#############"
selectk = SelectKBest(score_func=f_regression)
pipeLR = Pipeline([('selectk', selectk), ('LR', LogisticRegression(penalty="l1"))])
pipeLR, _,_,_,_  = do_classify(pipeLR, {"selectk__k":range(1,max_number_featrues+1), 
                                        "LR__C": [1e-3, 1e-2, 1e-1, 1, 1e2]}, 
                               dftouse,lcols, u'results',1, reuse_split=reuse_split)

print pipeLR.get_params()['LR__C']
print_result(pipeLR)
Long_ROI_result["log_regression"] = evaluate(pipeLR)[3]

#############====================== Log Regression =====================#############
1
Number of features: 7
Features: ['bb_pct', 'cci', 'rsi', 'ult_osc', 'willr', 'stoch_slowk', 'stoch_slowd']
train accuracy: 0.671701112878
test accuracy: 0.64192139738
ROI baseline: 0.123313
ROI long-only: 0.2238699919
ROI long-short: 0.3115179793
CPU times: user 4.77 s, sys: 71.4 ms, total: 4.84 s
Wall time: 5.07 s


In [43]:
%%time
print "#############====================== Linear SVM ========================#############"
clfsvm_b = Pipeline([('selectk', selectk), ('svm', LinearSVC(loss="hinge"))])
clfsvm_b,_,_,_,_  = do_classify(clfsvm_b, {"selectk__k":range(1,max_number_featrues+1), 
                                           "svm__C": [1e-300, 1e-200, 1e-100, 1e-10, 1e-1, 1]}, 
                                dftouse,lcols, u'results',1, reuse_split=reuse_split_balanced)

print clfsvm_b.get_params()['svm__C']
print_result(clfsvm_b)
Long_ROI_result["Linear_SVM"] = evaluate(clfsvm_b)[3]

#############====================== Linear SVM ========================#############
1
Number of features: 5
Features: ['bb_pct', 'cci', 'willr', 'stoch_slowk', 'stoch_slowd']
train accuracy: 0.662162162162
test accuracy: 0.64192139738
ROI baseline: 0.123313
ROI long-only: 0.1100111091
ROI long-short: 0.0808812102
CPU times: user 6.19 s, sys: 111 ms, total: 6.3 s
Wall time: 7.06 s


In [ ]:
%%time
print "#############====================== RBF SVM ===========================#############"
pipesvm2 = Pipeline([('selectk', selectk), ('svm2', SVC())])
pipesvm2,_,_,_,_  = do_classify(pipesvm2, {"selectk__k":range(1,max_number_featrues+1), 
                                            "svm2__C": [1e-100, 1e-10, 1e-1, 1, 1e10], 
                                            "svm2__gamma": [ 1e-9, 1e-10, 1e-11]}, 
                                 dftouse,lcols, u'results',1, reuse_split=reuse_split_balanced)
print pipesvm2.get_params()['svm2__C'], pipesvm2.get_params()['svm2__gamma']
print_result(pipesvm2)
Long_ROI_result["RBF_SVM"] = evaluate(pipesvm2)[3]

#############====================== RBF SVM ===========================#############
1e-100 1e-09
Number of features: 2
Features: ['bb_pct', 'stoch_slowk']
train accuracy: 0.664546899841
test accuracy: 0.646288209607
ROI baseline: 0.123313
ROI long-only: 0.3082939624
ROI long-short: 0.4945559228
CPU times: user 38.5 s, sys: 367 ms, total: 38.9 s
Wall time: 40.3 s


In [ ]:
%%time
print "#############====================== Random Forest =====================#############"
pipeRF = Pipeline([('selectk', selectk), ('RF', RandomForestClassifier())])
pipeRF,_,_,_,_  = do_classify(pipeRF, {"selectk__k": [5,6,7,8,9,10],
                                       "RF__max_depth": [3,5,7,10], 
                                       "RF__n_estimators": [5,10,20,40],
                                       "RF__max_features": [1,2,3,4,5]}, 
                              dftouse, lcols, u'results', 1, reuse_split=reuse_split)

print pipeRF.get_params()['RF__max_depth'], pipeRF.get_params()['RF__n_estimators'], pipeRF.get_params()['RF__max_features']
print_result(pipeRF)
Long_ROI_result["Random_forest"] = evaluate(pipeRF)[3]

In [ ]:
%%time
print "#############====================== Extra Trees= =====================#############"
pipeET = Pipeline([('selectk', selectk), ('ET', ExtraTreesClassifier())])
pipeET, _,_,_,_  = do_classify(pipeET, {"selectk__k": [5,6,7,8,9,10],
                                        "ET__max_depth": [3,5,7,10,15,25,50], 
                                        "ET__n_estimators": [5,10,20,40],
                                        "ET__max_features": [1,2,3,4,5]}, 
                               dftouse, lcols, u'results', 1, reuse_split=reuse_split)
print "ET__max_depth: {0}".format(pipeET.get_params()['ET__max_depth']) 
print "ET__n_estimators: {0}".format(pipeET.get_params()['ET__n_estimators']) 
print "ET__max_features: {0}".format(pipeET.get_params()['ET__max_features']) 
print_result(pipeET)
Long_ROI_result["Extra_Trees"] = evaluate(pipeET)[3]

In [ ]:
%%time
print "#############====================== Gaussian NB ==========================#############"
pipeNB = Pipeline([('selectk', selectk), ('NB', GaussianNB())])
pipeNB,_,_,_,_ = do_classify(pipeNB, {"selectk__k":range(1,max_number_featrues+1)},
                             dftouse, lcols, u'results',1, reuse_split=reuse_split)
print_result(pipeNB)
Long_ROI_result["Gaussian_NB"] = evaluate(pipeNB)[3]

In [ ]:
%%time
print "#############====================== Gradient Boosting ====================#############"
pipeGB = Pipeline([('selectk', selectk), ('GB', GradientBoostingClassifier())])
pipeGB, _,_,_,_  = do_classify(pipeGB, {"selectk__k":range(1,max_number_featrues+1), 
                                        "GB__n_estimators": [5,10,20,40],
                                        "GB__learning_rate": [0.1,0.5,1.0]}, 
                               dftouse,lcols, u'results',1, reuse_split=reuse_split)
print pipeGB.get_params()['GB__n_estimators'], pipeGB.get_params()['GB__learning_rate']
print_result(pipeGB)
Long_ROI_result["Gradient_bossting"] = evaluate(pipeGB)[3]

In [ ]:
print "ROI baseline: {0}".format(evaluate(pipeGB)[2])
Long_ROI_result